In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data_manager import DataManager
manager = DataManager(db_path='/home/jacob/Desktop/test_db.db', download_manager_workers=300) # master_db.db

In [4]:
manager.process_dataset("data_sets/image/llava_onevision.py", "/home/jacob/Desktop/data", dataset_subset="VizWiz(MathV360K)", sample=True, skip_file_download=False, batch_size=100)
# manager.file_manager.get_stats()
# len(manager.file_manager.get_failed_operations())

In [11]:
data = manager.sample_data(data_sources=["pixmo_point_explanations"], limit=3, offset=0)
# ['pixmo_docs_charts', 'pixmo_docs_diagrams', 'pixmo_docs_other']

data

In [3]:
manager.delete(data_sources=["llava_onevision_VizWiz(MathV360K)"])